In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder \
.appName('OlistData')\
.getOrCreate()

25/04/25 10:08:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [10]:
hdfs_path='/data/olist/'

In [11]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path +'product_category_name_translation.csv',header=True,inferSchema=True)

In [12]:
# Cache Frequently used Data for Better Performance
orders_df.cache()
customers_df.cache()
order_item_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [13]:
orders_items_joined_df=orders_df.join(order_item_df,'order_id','inner')

In [15]:
orders_items_products_df=orders_items_joined_df.join(products_df,'product_id','inner')

In [16]:
orders_items_products_sellers_df = orders_items_products_df.join(sellers_df,'seller_id','inner')

In [17]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

## Geolocation data 

In [19]:
full_orders_df=full_orders_df.join(geolocation_df,full_orders_df.customer_zip_code_prefix==geolocation_df.geolocation_zip_code_prefix,'left')

In [20]:
full_orders_df=full_orders_df.join(reviews_df,'order_id','left')

In [21]:
full_orders_df=full_orders_df.join(payments_df,'order_id','left')

In [22]:
full_orders_df.cache()

25/04/25 10:19:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [23]:
from pyspark.sql.functions import * 



In [24]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [25]:
## Total Revenues Per Seller
seller_revenue_df=full_orders_df.groupBy('seller_id').agg(sum('price'))

In [26]:
seller_revenue_df.show()

+--------------------+--------------------+
|           seller_id|          sum(price)|
+--------------------+--------------------+
|7a67c85e85bb2ce85...| 2.031279489000004E7|
|9d213f303afae4983...|   2321.400000000004|
|d2374cbcbb3ca4ab1...|   3375517.550000009|
|1835b56ce799e6a4d...|    6097995.11000001|
|d650b663c3b5f6fb3...|           2253869.1|
|aded58c8142dedc54...|   62478.00000000051|
|cca3071e3e9bb7d12...|1.0115934270000005E7|
|2b3e4a2a3ea8e0193...|  2094077.9600000014|
|dc4a0fc896dc34b0d...|           4300517.7|
|92eb0f42c21942b65...|   912998.0599999977|
|f3b80352b986ab4d1...|   1435817.200000001|
|e9779976487b77c6d...|   6293200.690000007|
|2a5b78b41cd05baea...|   4431339.830000005|
|d94a40fd42351c259...|           2164931.8|
|f63d8f293af3a6454...|   284921.5499999981|
|d71d863e5ef30d94e...|           1586336.9|
|5f2684dab12e59f83...|  1231050.6599999995|
|28f10b1c5e5abb9d4...|             72749.0|
|4e922959ae960d389...|   6884778.809999998|
|31561f325664a8a7a...|   297226.

#### Total Orders Per Custoemr
#### Average Review Score Per Seller
#### Most Sold Products ( Top 10 )
#### Top Custoemrs By Spending

## Optimized joins for data integration

In [31]:
from pyspark.sql.functions import * 


In [32]:
orders_items_joined_df=orders_df.join(order_item_df,'order_id','inner')

In [33]:
orders_items_products_df=orders_items_joined_df.join(products_df,'product_id','inner')

In [37]:
orders_items_products_sellers_df = orders_items_products_df.join(broadcast(sellers_df),'seller_id','inner')

In [38]:
full_orders_df=orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [40]:
# GEolocation Data
full_orders_df = full_orders_df.join(broadcast(geolocation_df),full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [42]:
full_orders_df=full_orders_df.join(broadcast(reviews_df),'order_id','left')

In [43]:
full_orders_df=full_orders_df.join(payments_df,'order_id','left')

In [44]:
full_orders_df.cache()

25/04/25 10:35:51 WARN CacheManager: Asked to cache already cached data.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

## Aggregation

### Total orders per customers

In [47]:
customer_order_count_df=full_orders_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'))\
.orderBy(desc('total_orders'))
customer_order_count_df.show(5)

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
+--------------------+------------+
only showing top 5 rows



## Average Review Score PerSeller 

In [49]:
seller_review_df=full_orders_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy(desc('avg_review_score'))
seller_review_df.show()

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|4e92348462c75b6c8...|             5.0|
|ebd1417732defaf66...|             5.0|
|d263fa444c1504a75...|             5.0|
|804287717b9156fb7...|             5.0|
|10264f60a8f0a4d2f...|             5.0|
|d3891911c2feae53c...|             5.0|
|7238f877570096ae4...|             5.0|
|44717f64ec2a45797...|             5.0|
|64cc804291f90c81f...|             5.0|
|e9c2dc0dd4ac38448...|             5.0|
|186cdd1b2df32caa7...|             5.0|
|b5dd7151a92ccaaa2...|             5.0|
|dd39ad39b5d3df330...|             5.0|
|c8c1bea22194a4eef...|             5.0|
|ce4755084bc097113...|             5.0|
|09bad886111255c5b...|             5.0|
|666658b8da8370f30...|             5.0|
|d598f929fc44e1e38...|             5.0|
|3289d8c3eef4873f9...|             5.0|
|a64e44665225d19df...|             5.0|
+--------------------+----------------+
only showing top 20 rows



In [50]:
# Top 10 Most Sold Products
top_products_df = full_orders_df.groupBy('product_id')\
.agg(count('order_id').alias('total_sold'))\
.orderBy(desc('total_sold'))\
.limit(10)
top_products_df.show(5)

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     86740|
|422879e10f4668299...|     81110|
|99a4788cb24856965...|     78775|
|389d119b48cf3043d...|     60248|
|d1c427060a0f73f6b...|     59274|
+--------------------+----------+
only showing top 5 rows



### Window function aand Ranking

In [52]:
from pyspark.sql.window import Window

## Dense Rank for sellers based on revenues

In [54]:
window_spec =Window.partitionBy('seller_id').orderBy(desc('price'))

In [55]:
# Rank Top Selling Products Per seller
top_seller_products_df = full_orders_df.withColumn('rank',rank().over(window_spec)).filter(col('rank')<=5)

In [56]:
top_seller_products_df.select('seller_id','price','rank').show()

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 20 rows



## Advance Aggregation and Enrichment 

In [58]:
full_orders_df

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [59]:
# Total Revenue & Average Order Value (AOV) per customer
customer_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
count('order_id').alias('total_orders'),
sum('price').alias('total_spent'),
round(avg('price'),2).alias('AOV')
)\
.orderBy(desc('total_spent'))
customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [60]:
# seller performance metrices( revenue,average review ,order count)
seller_performance_df=full_orders_df.groupBy('seller_id') \
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('review_score'),2).alias('avg_review_score'),
    round(stddev('price'),2).alias('price_variability')
)\
.orderBy(desc('total_revenue'))

In [61]:
seller_performance_df.show()

+--------------------+------------+--------------------+----------------+-----------------+
|           seller_id|total_orders|       total_revenue|avg_review_score|price_variability|
+--------------------+------------+--------------------+----------------+-----------------+
|4869f7a5dfa277a7d...|      184587| 3.613871731999996E7|            4.09|           111.65|
|53243585a1d6dc264...|       54514| 3.429159294999997E7|            4.12|           499.65|
|4a3ca9315b744ce9f...|      330661|  3.37595708400001E7|            3.77|            59.37|
|7c67e1448b00f6e96...|      233306| 3.228232178999976E7|            3.42|            50.39|
|fa1c13f2614d7b5c4...|       87686| 3.013938630999999E7|            4.38|            307.7|
|da8622b14eb17ae28...|      264433|2.9857669730000038E7|            3.98|            72.92|
|7e93a43ef30c4f03f...|       50226|2.6315706299999937E7|            4.15|           377.24|
|1025f0e2d44d7041d...|      229587| 2.293751851999999E7|            3.89|       

In [64]:
#Product Popularity Metrices 
product_metrices_df=full_orders_df.groupBy('product_id')\
.agg(count('order_id').alias('total_sales'),
     sum('price').alias('total_revenue'),
     round(avg('price'),2).alias('avg_price'),
     round(stddev('price'),2).alias('price_volatility'),\
     collect_set('seller_id').alias('unique_sellers')
)\
.orderBy(desc('total_sales'))
     
                                 

In [65]:
product_metrices_df.show(5)

+--------------------+-----------+------------------+---------+----------------+--------------------+
|          product_id|total_sales|     total_revenue|avg_price|price_volatility|      unique_sellers|
+--------------------+-----------+------------------+---------+----------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740|6164630.3000000175|    71.07|            3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.510000013|    54.77|            4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775| 6921762.710000026|    87.87|            4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|3280533.1300000106|    54.45|            4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274| 8220103.329999983|   138.68|           16.58|[a1043bafd471dff5...|
+--------------------+-----------+------------------+---------+----------------+--------------------+
only showing top 5 rows



In [67]:
customer_retention_df.show()

+--------------------+-------------------+-------------------+------------+------+
|         customer_id|   first_order_date|    last_order_date|total_orders|   aov|
+--------------------+-------------------+-------------------+------------+------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427| 85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752| 43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|  26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001| 36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876| 12.49|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876| 969.0|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|  59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|  34.9|
|24e7dc2ff8c071263...|2017-11-24 16:16:45|2017-11-24 16:16:45|        6597|  59.2|
|7bb

In [69]:
customer_retention_df.show(5)

+--------------------+-------------------+-------------------+------------+-----+
|         customer_id|   first_order_date|    last_order_date|total_orders|  aov|
+--------------------+-------------------+-------------------+------------+-----+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427|85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752|43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556| 26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001|36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876|12.49|
+--------------------+-------------------+-------------------+------------+-----+
only showing top 5 rows



In [73]:
## Extended  Enrichment

In [74]:
##Order status flags

In [75]:
full_orders_df=full_orders_df.withColumn('is_delivered',when(col('order_status')=='delivered',lit(1)).otherwise(lit(0)))\
.withColumn('is_canceled',when(col('order_status')=='canceled',lit(1)).otherwise(lit(0)))
            

In [76]:
full_orders_df.where(full_orders_df['order_status']=='canceled').select('order_status','is_delivered','is_canceled').show(50)

+------------+------------+-----------+
|order_status|is_delivered|is_canceled|
+------------+------------+-----------+
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|


In [77]:
## Order Revenue Collection
full_orders_df=full_orders_df.withColumn('order_revenue',col('price')+col('freight_value'))

In [78]:
full_orders_df.select('price','freight_value','order_revenue').show()

+-----+-------------+------------------+
|price|freight_value|     order_revenue|
+-----+-------------+------------------+
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
+-----+-------------+------------------+
only showing top

In [79]:
customer_spending_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- total_orders: long (nullable = false)
 |-- total_spent: double (nullable = true)
 |-- AOV: double (nullable = true)



In [80]:
# Customer Segmentation based on spending
customer_spending_df = customer_spending_df.withColumn(
'customer_segment',
when(col('AOV') >=1200,"High-Value")
.when( (col('AOV')<1200) & (col('AOV') >=700),'Medium_Value')
.otherwise('Low-Value'))

In [83]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|       total_spent|    AOV|customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_Value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High-Value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|      High-Value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High-Value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low-Value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High-Value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_Value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High-Value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High-Value|
|695476b5848d64ba0...|         687|18205

In [84]:
full_orders_df=full_orders_df.join(customer_spending_df.select('customer_id','customer_segment'),'customer_id',how='left')

In [85]:
full_orders_df.select('customer_id','customer_segment').show()

+--------------------+----------------+
|         customer_id|customer_segment|
+--------------------+----------------+
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
|deffd3aca3c7bd8f0...|       Low-Value|
+--------------------+----------------+
only showing top 20 rows



In [86]:
full_orders_df.select('order_purchase_timestamp').show()

+------------------------+
|order_purchase_timestamp|
+------------------------+
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
+------------------------+
only showing top 20 rows



In [87]:
#hourly order distribution
full_orders_df=full_orders_df.withColumn('hour_of_day',expr('hour(order_purchase_timestamp)'))

In [88]:
full_orders_df.select('order_purchase_timestamp','hour_of_day').show()

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
+------------------------+-----------+
only showing top 20 rows



In [93]:
from pyspark.sql.functions import dayofweek, when, lit

full_orders_df = full_orders_df.withColumn(
    'order_day_type',
    when(dayofweek('order_purchase_timestamp').isin(1, 7), lit('Weekend'))
    .otherwise(lit('Weekday'))
)


In [94]:
full_orders_df.select('order_purchase_timestamp','order_day_type').show()

+------------------------+--------------+
|order_purchase_timestamp|order_day_type|
+------------------------+--------------+
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
|     2018-08-07 23:19:16|       Weekday|
+------------------------+--------

In [95]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [96]:
!hadoop fs -mkdir /olist/processed/

mkdir: `hdfs://cluster-0c12-m/olist': No such file or directory


In [97]:
full_orders_df.write.mode('overwrite').parquet('/olist/processed')

In [98]:
!hadoop fs -ls -h /olist/processed/

Found 11 items
-rw-r--r--   2 root hadoop          0 2025-04-25 11:49 /olist/processed/_SUCCESS
-rw-r--r--   2 root hadoop      8.9 M 2025-04-25 11:49 /olist/processed/part-00000-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      7.7 M 2025-04-25 11:48 /olist/processed/part-00001-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      7.5 M 2025-04-25 11:48 /olist/processed/part-00002-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.3 M 2025-04-25 11:49 /olist/processed/part-00003-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.1 M 2025-04-25 11:49 /olist/processed/part-00004-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      8.2 M 2025-04-25 11:49 /olist/processed/part-00005-f7a04f7f-63a5-4933-8628-a9908b548edd-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.2 M 2025-04-25 11:49 /olist/pr

In [101]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f46896a6550>>